# Forward Chaining

This Jupyter notebook demonstrates how *Forward Chaining* can be used to infere knowledge from a First-Order Logic knowledge base.

Import modules:

In [1]:
from utils import *
from logic import *

### Initialize Knowledge Base:

We consider the *Criminal West* example from the lecture. The example makes use of the following predicate symbols:

* `Criminal(x)`: `x` is a criminal
* `American(x)`: `x` is an American
* `Enemy(x ,y)`: `x` is an enemy of `y`
* `Sells(x ,y, z)`: `x` sells `y` to `z`
* `Owns(x ,y)`: `x` owns `y`
* `Weapon(x)`: `x` is a weapon
* `Hostile(x)`: `x` is a hostile nation

Furthermore, the constant symbols `Nono`, `West`, and `America` are given.

Before we can apply the *Forward Chaining* algorithm, we have to construct our knowledge base by adding all relevant clauses:

In [2]:
clauses = []

<em>“... it is a crime for an American to sell weapons to hostile nations”</em><br/>

$\text{American}(x) \land \text{Weapon}(y) \land \text{Sells}(x, y, z) \land \text{Hostile}(z) \implies \text{Criminal} (x)$

In [3]:
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))

<em>"Nono ... has some missiles"</em><br/>
This states the existence of some missile which is owned by Nono. $\exists x \text{Owns}(\text{Nono}, x) \land \text{Missile}(x)$. We invoke existential instantiation to introduce a new constant `M1` which is the missile owned by Nono.

$\text{Owns}(\text{Nono}, \text{M1}), \text{Missile}(\text{M1})$

In [4]:
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))

<em>".. all of its missiles were sold to it by Colonel West"</em><br/>

$\text{Missile}(x) \land \text{Owns}(\text{Nono}, x) \implies \text{Sells}(\text{West}, x, \text{Nono})$

In [5]:
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))

<em>“Missiles are weapons”</em><br/>

$\text{Missile}(x) \implies \text{Weapon} (x)$

In [6]:
clauses.append(expr("Missile(x) ==> Weapon(x)"))

<em>“An enemy of America counts as hostile”</em><br/>

$\text{Enemy}(x, \text{America}) \implies \text{Hostile}(x)$

In [7]:
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))

<em>"West, who is American ..."</em><br/>

$\text{American}(\text{West})$

In [8]:
clauses.append(expr("American(West)"))

<em>"The country Nono, an enemy of America ..."</em><br/>

$\text{Enemy}(\text{Nono}, \text{America})$

In [9]:
clauses.append(expr("Enemy(Nono, America)"))

Now that we have converted the information into first-order definite clauses we can create our first-order logic knowledge base.

In [10]:
crime_kb = FolKB(clauses)

### Forward Chaining Algorithm:

Now we want to check if Colonel West is a criminal by applying the *Forward Chaining* algorithm. We use the following modified version of the `fol_fc_ask` function which additionally prints out the infered clauses:

In [11]:
def fol_fc_ask_debug(KB, alpha):
    """A simple forward-chaining algorithm. (Debug Version)"""
    print("Forward Chaining:")
    print(KB.clauses, "==>", alpha)
    print()
    
    #all constant symbols (like 'Nono' or 'West')
    kb_consts = list({c for clause in KB.clauses for c in constant_symbols(clause)})
    
    # iterator over all possible substitutions
    def enum_subst(p):
        query_vars = list({v for clause in p for v in variables(clause)})
        for assignment_list in itertools.product(kb_consts, repeat=len(query_vars)):
            theta = {x: y for x, y in zip(query_vars, assignment_list)}
            yield theta

    # check if we can answer without new inferences
    for q in KB.clauses:
        phi = unify(q, alpha, {})
        if phi is not None:
            yield phi

    while True:
        # new clauses to be added to KB
        new = []
        for rule in KB.clauses:
            # split rule into a list of premises (p), and a conclusion (q)
            p, q = parse_definite_clause(rule)
            for theta in enum_subst(p):
                if set(subst(theta, p)).issubset(set(KB.clauses)):
                    # all substituted premises are in the knowledge base
                    # => substituted conclusion q_ is true
                    q_ = subst(theta, q)
                    if all([unify(x, q_, {}) is None for x in KB.clauses + new]):
                        # if there is not a more generic clause add q_ to new
                        new.append(q_)
                        phi = unify(q_, alpha, {})
                        if phi is not None:
                            # yield substitution for alpha
                            yield phi
        if not new:
            # no new clauses, so stop
            break
        for clause in new:
            print("add to KB:", clause)
            KB.tell(clause)
    print()
    return None

To check if Colonel West is a criminal we apply the *Forward Chaining* algorithm for the query $\alpha = \text{Criminal(West)}$:

In [12]:
answer = fol_fc_ask_debug(crime_kb, expr('Criminal(West)'))
print(list(answer))

Forward Chaining:
[((((American(x) & Weapon(y)) & Sells(x, y, z)) & Hostile(z)) ==> Criminal(x)), Owns(Nono, M1), Missile(M1), ((Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)), (Missile(x) ==> Weapon(x)), (Enemy(x, America) ==> Hostile(x)), American(West), Enemy(Nono, America)] ==> Criminal(West)

add to KB: Sells(West, M1, Nono)
add to KB: Weapon(M1)
add to KB: Hostile(Nono)
add to KB: Criminal(West)

[{}]
